In [35]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
from llm.prompt_loader import PromptLoader
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from llm.ml_tools import svd_embed_score
from sklearn.decomposition import PCA
from metric_utils import get_measures, print_measures
from linear_probe import get_linear_acc
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [36]:
data_reader=ReadData('/home/hallscope/output/pope')
data=data_reader.read_all()
data[0]

{'img_path': '/root/autodl-fs/coco_images/val/COCO_val2014_000000310196.jpg',
 'question': 'Is there a snowboard in the image?\nAnswer the question using a single word or phrase.',
 'label': 1,
 'question_id': 1,
 'category': 'popular',
 'most_likely': {'embedding': array([[[ 3.8909912e-03,  1.4953613e-03,  5.4626465e-03, ...,
           -4.2114258e-03,  1.5075684e-02,  2.4261475e-03]],
  
         [[-5.3787231e-03, -1.2474060e-03,  8.8424683e-03, ...,
           -1.1787415e-02,  8.6975098e-03,  9.3383789e-03]],
  
         [[-3.2531738e-02,  3.3264160e-03,  1.9363403e-02, ...,
            8.7356567e-03,  2.6062012e-02,  9.3231201e-03]],
  
         ...,
  
         [[ 1.5664062e+00,  2.2695312e+00, -4.0673828e-01, ...,
           -2.7734375e+00, -1.3789062e+00, -2.1718750e+00]],
  
         [[ 2.2363281e+00,  2.5859375e+00, -5.2978516e-01, ...,
           -2.6347656e+00, -1.5664062e+00, -2.3593750e+00]],
  
         [[ 8.9501953e-01,  3.9526367e-01,  9.9853516e-01, ...,
           -5.

In [37]:
df=pd.DataFrame(data)

In [38]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a snowboard in the image?\nAnswer the...,1,1,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a dining table in the image?\nAnswer ...,0,2,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a person in the image?\nAnswer the qu...,1,3,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a car in the image?\nAnswer the quest...,0,4,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,/root/autodl-fs/coco_images/val/COCO_val2014_0...,Is there a skis in the image?\nAnswer the ques...,1,5,popular,{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [39]:
gt_label=df['label'].to_numpy()

In [40]:
length=len(data)
wild_ratio=0.9

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    wild_ratio * length)]  # trainset and validation index

test_idxs=all_indices[int(
    wild_ratio * length):]  #test index

validset_len = 1500
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
gt_label_test = []
gt_label_wild = []
gt_label_val = []

for i in range(length):
    if i not in train_val_idxs:
        gt_label_test.extend(gt_label[i: i+1])
    elif i in train_idxs:
        gt_label_wild.extend(gt_label[i: i+1])
    else:
        gt_label_val.extend(gt_label[i: i+1])

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
gt_label_test = gt_label[test_idxs]
gt_label_wild = gt_label[train_idxs]
gt_label_val = gt_label[val_idxs]

print(f'trainset length:{len(gt_label_wild)}')
print(f'validset length:{len(gt_label_val)}')
print(f'testset length:{len(gt_label_test)}')
print(f'trainset postive: {sum(gt_label_wild==1)} negative: {sum(gt_label_wild==0)}')
print(
    f'testset postive: {sum(gt_label_test==1)} negative: {sum(gt_label_test==0)}')

trainset length:1200
validset length:1500
testset length:300
trainset postive: 589 negative: 611
testset postive: 145 negative: 155


In [41]:
embed_generated = df['most_likely'].apply(lambda x: np.squeeze(x['embedding'])).to_list()
embed_generated =np.stack(embed_generated)
embed_generated.shape

(3000, 33, 4096)

In [42]:
embed_generated_wild = embed_generated[train_idxs][:, 1:, :]
embed_generated_eval = embed_generated[val_idxs][:, 1:, :]

In [43]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [44]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [45]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embed_generated_eval.shape[1])
}
grid_search=GridSearch(evaluator,grid,embed_generated_eval,gt_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer
found best n_components:1,layer:0 Score: 0.49121555765824093
found best n_components:1,layer:1 Score: 0.4963770178072153
found best n_components:1,layer:2 Score: 0.5189490683760076
found best n_components:1,layer:3 Score: 0.519904169719908
found best n_components:1,layer:5 Score: 0.5217076571737538
found best n_components:1,layer:17 Score: 0.8614782905642391
found best n_components:1,layer:20 Score: 0.8810338571865596
found best n_components:1,layer:21 Score: 0.9026490278242185
found best n_components:1,layer:22 Score: 0.9275439133187726
found best n_components:1,layer:26 Score: 0.9345764472364312
found best n_components:1,layer:27 Score: 0.9445721074835836
found best n_components:1,layer:30 Score: 0.9472364311580026


In [46]:
best_paras

{'n_components': 1, 'layer': 30, 'best_score': np.float64(0.9472364311580026)}

In [48]:
best_layer=best_paras['layer']

In [49]:
discriminator=PCADiscriminator(best_paras['n_components'],embed_generated_eval[:,best_layer,:])
best_split=discriminator.get_best_split(gt_label_val)

In [50]:
best_split

np.float32(32236.715)

In [51]:
discriminator=PCADiscriminator(best_paras['n_components'],embed_generated_wild[:,best_layer,:])
scores=discriminator.get_score()

In [52]:
preds=(scores>best_split)
preds

array([False, False,  True, ..., False, False,  True])

In [53]:
accuracy_score(gt_label_wild,preds)

0.8558333333333333

In [54]:
roc_auc_score(gt_label_wild,scores)

np.float64(0.9371149747554038)

In [ ]:
pca_model = PCA(n_components=best_parameters['k'], whiten=False).fit(
    embed_generated_wild[:, best_parameters['best_layer'], :])
projection = pca_model.components_.T
if weighted_svd:
    projection = pca_model.singular_values_ * projection
scores = np.mean(np.matmul(
    embed_generated_wild[:, best_parameters['best_layer'], :], projection), -1, keepdims=True)
assert scores.shape[1] == 1
best_scores = np.sqrt(np.sum(np.square(scores), axis=1)
                      ) * best_parameters['best_sign']

'''get score for direct projection'''
# direct projection

embed_generated_test = embed_generated[test_idxs][:, 1:, :]

test_scores = np.mean(np.matmul(embed_generated_test[:, best_parameters['best_layer'], :],
                                projection), -1, keepdims=True)

assert test_scores.shape[1] == 1
test_scores = np.sqrt(np.sum(np.square(test_scores), axis=1))

measures = get_measures(best_parameters['best_sign'] * test_scores[gt_label_test == 1],
                        best_parameters['best_sign'] * test_scores[gt_label_test == 0], plot=False)
print_measures(measures[0], measures[1], measures[2], 'direct-projection')

In [11]:
print(f'best_score distributes in range: {best_scores.max()}-{best_scores.min()}')

best_score distributes in range: 8639.2412109375-1.1222127676010132


In [12]:
feat_indices_test = []
for i in range(length):
    if i not in train_val_idxs:
        feat_indices_test.extend(np.arange(1 * i, 1 * i + 1).tolist())

embed_generated_test = embed_generated[feat_indices_test][:, 1:, :]

In [13]:
thresholds = np.linspace(0, 1, num=40)[1:-1]

# graid search
auroc_over_thres = []
for thres_wild in tqdm(thresholds):
    best_auroc = 0
    best_accuracy=0
    for layer in range(len(embed_generated_wild[0])):
        thres_wild_score = np.sort(best_scores)[
            int(len(best_scores) * thres_wild)]
        true_wild = embed_generated_wild[:, layer,:][best_scores > thres_wild_score]
        false_wild = embed_generated_wild[:, layer,:][best_scores <= thres_wild_score]

        embed_train = np.concatenate([true_wild, false_wild], 0) # true label: 1; false label:0
        label_train = np.concatenate([np.ones(len(true_wild)),
                                      np.zeros(len(false_wild))], 0)

        # gt training, saplma
        # embed_train = embed_generated_wild[:,layer,:]
        # label_train = gt_label_wild
        # gt training, saplma
        print(f'postive: {sum(label_train==1)} negative: {sum(label_train==0)}')

        best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            embed_train,
            label_train,
            embed_train,
            label_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

        clf.eval()
        output = clf(torch.from_numpy(
            embed_generated_test[:, layer, :]).to(torch.float32).cuda())
        pca_wild_score_binary_cls = torch.sigmoid(output)

        pca_wild_score_binary_cls = pca_wild_score_binary_cls.cpu().data.numpy()

        #pred_labels=(pca_wild_score_binary_cls >= 0.5).astype(int)
        
        #pred_acc=accuracy_score(gt_label_test,pred_labels)

        if np.isnan(pca_wild_score_binary_cls).sum() > 0:
            breakpoint()
        measures = get_measures(pca_wild_score_binary_cls[gt_label_test == 1],
                                pca_wild_score_binary_cls[gt_label_test == 0], plot=False)
        
        #auroc=roc_auc_score(gt_label_test,pca_wild_score_binary_cls)
        #if auroc> best_auroc:
        #    best_auroc=auroc
        #    best_accuracy=pred_acc
        #    best_layer=layer

        #if pred_acc> best_accuracy:
        #    best_accuracy=pred_acc
        
        
        if measures[0] > best_auroc:
            best_auroc = measures[0]
            best_result = [100 * measures[0]]

            fpr, tpr, thresholds = roc_curve(gt_label_test,pca_wild_score_binary_cls)

            # Calculate Youden's J statistic
            youdens_j = tpr - fpr
            # Find the index of the maximum J statistic 
            best_index = np.argmax(youdens_j) 
            best_threshold = thresholds[best_index]

            pred_labels=(pca_wild_score_binary_cls >= best_threshold).astype(int)
            best_accuracy=accuracy_score(gt_label_test,pred_labels)

            best_layer = layer


    print('thres: ', np.sort(best_scores)[
            int(len(best_scores) * thres_wild)], 'best result: ',
          best_result, 'best_layer: ', best_layer,'best_accuracy',best_accuracy,'linear best_threshold',best_threshold)

  0%|          | 0/38 [00:00<?, ?it/s]

postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62
postive: 2338 negative: 62


  0%|          | 0/38 [00:48<?, ?it/s]


KeyboardInterrupt: 